https://www.kaggle.com/ayuraj/train-covid-19-detection-using-yolov5

In [ ]:
!nvidia-smi

In [210]:
#config consts
DATASET =     {'train':'train.tar.gz',
               'test':'test.tar.gz',
               'train_mask':'train_mask.tar.gz'}

METADATA=     {'image_level':'train_image_level.csv',
               'study_level':'train_study_level.csv',
               'df_meta':'df_meta.csv',
               'df_fold_rand830':'df_fold_rand830.csv',
               'train_dup':'duplicate.txt',
               'sample_sub':'sample_submission.csv',
               'df_annotate':'df_annotate.csv',
               'df_meta_hw':'df_meta_hw.csv',
               'meta':'meta.csv'}

CFGMODEL_DIR_DICT = {'B3_512':'effb3-full-512-mask-v8/',
                     'B3_512_PNG': 'effb3-512-png-mask/',
                     'B4_512':'effb4-full-512-mask/',
                     'B5_600':'effb5-600-mask/',
                     'B5_640':'effb5-640-mask/',
                     'B5_640_M2': 'effb5-640-mask/',
                     'D121_640':'d121-640-mask/',
                     'D201_640':'d201-640-mask/',
                     'YOLO5_640':'custom-yolo5-640-v4/'}

INPUT_DIR ='/content/drive/My\ Drive/kaggle/covid19-det/input/'

OUTPUT_DIR = {'BSL':'/content/drive/My\ Drive/kaggle/covid19-det/output/',
              'NORM':'/content/drive/My Drive/kaggle/covid19-det/output/'}

IMPORT_DIR = '/content/drive/My Drive/kaggle/covid19-det/nbs/py/'

HENGCK_IM_DIR=IMPORT_DIR+'hengck_code/dummy_01q/'
HENGCK_IM_DIR_YOLO=IMPORT_DIR+'hengck_code/dummy_01a/'


WORK_DIR='/content/'

DATASET_DIR_DICT = {'256': INPUT_DIR+'256_jpg/',
                    '512': INPUT_DIR+'512_jpg/',
                    '512_PNG': INPUT_DIR+'512_png/',
                    '600': INPUT_DIR+'600_jpg/',
                    '640': INPUT_DIR+'640_jpg/',
                    '640_M2':INPUT_DIR+'640_jpg_m2/'}

EXPERIMENT='SZ640_AYURAJ_YOLOV5X_0FOLD'
EXPERIMENT_DIR = OUTPUT_DIR['BSL'] + EXPERIMENT+'/'
CFGMODEL_DIR = CFGMODEL_DIR_DICT['YOLO5_640']
MASK_SIZE=(40,40)
L1_FACTOR=4
DATASET_DIR = DATASET_DIR_DICT['640']
METADATA_DIR = INPUT_DIR+'metadata/'
FOLDS_SET=[0]
INITIAL_CHECKPOINTS=[None for i in range(5)]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def copy_dataset(ds_dict, ds_dir, work_dir):
  for record in ds_dict:
    print('copy', ds_dir+ds_dict[record], ' to', work_dir)
    !cp {ds_dir+ds_dict[record]} {work_dir}
    print('mkdir',work_dir+record)
    !mkdir {work_dir+record}
    print ('tar -xzf',work_dir+ds_dict[record],'-C',work_dir+record)
    !tar -xzf  {work_dir+ds_dict[record]} -C {work_dir+record}
    print ('rm ',work_dir+ds_dict[record])
    !rm {work_dir+ds_dict[record]}
def copy_metadata(md_dict,md_dir,work_dir):
  for record in md_dict:
    print('copy ', md_dir+md_dict[record],' to ',work_dir)
    !cp {md_dir+md_dict[record]} {work_dir}
copy_dataset(DATASET,DATASET_DIR, WORK_DIR)
copy_metadata(METADATA,METADATA_DIR,WORK_DIR)
print('copy', DATASET_DIR+METADATA['meta'], ' to', WORK_DIR)
!cp { DATASET_DIR+METADATA['meta']} {WORK_DIR}
print('mkdir '+EXPERIMENT_DIR)
!mkdir {EXPERIMENT_DIR}
!ls /content/

copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train.tar.gz  to /content/
mkdir /content/train
mkdir: cannot create directory ‘/content/train’: File exists
tar -xzf /content/train.tar.gz -C /content/train
rm  /content/train.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/test.tar.gz  to /content/
mkdir /content/test
mkdir: cannot create directory ‘/content/test’: File exists
tar -xzf /content/test.tar.gz -C /content/test
rm  /content/test.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train_mask.tar.gz  to /content/
mkdir /content/train_mask
mkdir: cannot create directory ‘/content/train_mask’: File exists
tar -xzf /content/train_mask.tar.gz -C /content/train_mask
rm  /content/train_mask.tar.gz
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_image_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_study_level.csv  to  /content/
copy  /content/drive/My\ Drive/

In [ ]:
def draw_batch(imbatch):
  bs = imbatch.shape[0]
  fig, axs = plt.subplots(1, bs, figsize=(30, 30))
  for i in range(bs):
    axs[i].imshow(imbatch[i,0,:,:],cmap='gray')
  plt.show()

In [ ]:
TMP_DIR='/content/tmp'

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 8333, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 8333 (delta 22), reused 24 (delta 13), pack-reused 8288
Receiving objects: 100% (8333/8333), 9.21 MiB | 6.77 MiB/s, done.
Resolving deltas: 100% (5764/5764), done.


In [ ]:
%cd yolov5

/content/yolov5/yolov5


In [ ]:
%pip install -qr requirements.txt  # install dependencies

In [ ]:
%cd ../

/content


In [ ]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla V100-SXM2-16GB)


In [ ]:
# Install W&B 
!pip install -q --upgrade wandb
# Login 
import wandb
wandb.login()

wandb: Currently logged in as: diwert (use `wandb login --relogin` to force relogin)


True

In [ ]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
TRAIN_PATH = '/content/train/'
IMG_SIZE = 640
BATCH_SIZE = 24
EPOCHS = 35

In [ ]:
def read_list_from_file(list_file, comment='#'):
    with open(list_file) as f:
        lines  = f.readlines()
    strings=[]
    for line in lines:
        if comment is not None:
            s = line.split(comment, 1)[0].strip()
        else:
            s = line.strip()
        if s != '':
            strings.append(s)
    return strings
    
def get_folds(data_dir):
  df_study = pd.read_csv(data_dir+'/train_study_level.csv')
  df_fold  = pd.read_csv(data_dir+'/df_fold_rand830.csv')
  df_meta  = pd.read_csv(data_dir+'/df_meta.csv')

  df_study.loc[:, 'id'] = df_study.id.str.replace('_study', '')
  df_study = df_study.rename(columns={'id': 'study_id'})

          
  df = df_study.copy()
  df = df.merge(df_fold, on='study_id')
  df = df.merge(df_meta, left_on='study_id', right_on='study')

  duplicate = read_list_from_file(data_dir + '/duplicate.txt')
  df = df[~df['image'].isin(duplicate)].reset_index(drop=True)
       
  return df

In [ ]:
# Load image level csv file
df = pd.read_csv('/content/train_image_level.csv')

# Modify values in the id column
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)
# Add absolute path
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.jpg', axis=1)
# Get image level labels
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)

df.head(1)

,id,boxes,label,StudyInstanceUID,path,image_level
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,/content/train/000a312787f2.jpg,opacity


In [ ]:
# Load meta.csv file
# Original dimensions are required to scale the bounding box coordinates appropriately.
meta_df = pd.read_csv('/content/meta.csv')
train_meta_df = meta_df.loc[meta_df.split == 'train']
train_meta_df = train_meta_df.drop('split', axis=1)
train_meta_df.columns = ['id', 'dim0', 'dim1']

train_meta_df.head(1)

,id,dim0,dim1
1263,d8ba599611e5,2336,2836


In [ ]:
df = df.merge(train_meta_df, on='id',how="left")
df.head(1)

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,/content/train/000a312787f2.jpg,opacity,3488,4256


In [ ]:
duplicate = read_list_from_file('/content/duplicate.txt')
df = df[~df['id'].isin(duplicate)].reset_index(drop=True)
df_h  = get_folds('/content')
df_h.rename(columns={'image': 'id'}, inplace=True)
df = df.merge(df_h[['fold','id']],on='id',how='left')
df.head(1)

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1,fold
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,/content/train/000a312787f2.jpg,opacity,3488,4256,2


In [ ]:
# Create train and validation split.

train_df = df[df['fold']!=fold].reset_index(drop=True)
valid_df = df[df['fold']==fold].reset_index(drop=True)

train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

df['split']=df['fold'].apply(lambda x: 'train' if x!=fold else 'valid' )
df.head(1)

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1,fold,split
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,/content/train/000a312787f2.jpg,opacity,3488,4256,2,train


In [ ]:
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')

Size of dataset: 6227, training images: 4980. validation images: 1247


In [ ]:
os.makedirs('/content/tmp/covid/images/train', exist_ok=True)
os.makedirs('/content/tmp/covid/images/valid', exist_ok=True)

os.makedirs('/content/tmp/covid/labels/train', exist_ok=True)
os.makedirs('/content/tmp/covid/labels/valid', exist_ok=True)

! ls tmp/covid/images

train  valid


In [ ]:
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'tmp/covid/images/train/{row.id}.jpg')
    else:
        copyfile(row.path, f'tmp/covid/images/valid/{row.id}.jpg')

100%|██████████| 6227/6227 [00:02<00:00, 2400.75it/s]


In [ ]:
!ls '/content/tmp/covid/labels/train/' -l | wc

   3436   30917  195807


In [ ]:
!ls '/content/tmp/covid/labels/valid/' -l | wc

    859    7724   48917


In [ ]:
import yaml

data_yaml = dict(
    train = '/content/tmp/covid/images/train',
    val = '/content/tmp/covid/images/valid',
    nc = 2,
    names = ['none', 'opacity']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('/content/yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat /content/yolov5/data/data.yaml

{names: [none, opacity], nc: 2, train: /content/tmp/covid/images/train, val: /content/tmp/covid/images/valid}


In [ ]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

In [ ]:
df

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1,fold,split
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,/content/train/000a312787f2.jpg,opacity,3488,4256,2,train
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,/content/train/000c3a3f293f.jpg,none,2320,2832,3,train
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,/content/train/0012ff7358bc.jpg,opacity,2544,3056,1,train
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,/content/train/001398f4ff4f.jpg,opacity,3520,4280,0,valid
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,/content/train/001bd15d1891.jpg,opacity,2800,3408,4,train
...,...,...,...,...,...,...,...,...,...,...
6222,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,/content/train/ffcc6edd9445.jpg,none,4240,3480,3,train
6223,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,/content/train/ffd91a2c4ca0.jpg,none,2800,3408,4,train
6224,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,/content/train/ffd9b6cf2961.jpg,opacity,2388,3050,2,train
6225,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,/content/train/ffdc682f7680.jpg,opacity,3488,4256,4,train


In [ ]:
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.id
    # Get split
    split = row.split
    # Get image-level label
    label = row.image_level
    
    if row.split=='train':
        file_name = f'/content/tmp/covid/labels/train/{row.id}.txt'
    else:
        file_name = f'/content/tmp/covid/labels/valid/{row.id}.txt'
        
    
    if label=='opacity':
        # Get bboxes
        bboxes = get_bbox(row)
        # Scale bounding boxes
        scale_bboxes = scale_bbox(row, bboxes)
        # Format for YOLOv5
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
        with open(file_name, 'w') as f:
            for bbox in yolo_bboxes:
                bbox = [1]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')

100%|██████████| 6227/6227 [00:02<00:00, 2197.00it/s]


In [ ]:
%cd /content/yolov5/

/content/yolov5


In [ ]:
--img {IMG_SIZE} \ # Input image size.
--batch {BATCH_SIZE} \ # Batch size
--epochs {EPOCHS} \ # Number of epochs
--data data.yaml \ # Configuration file
--cfg models/yolov5x.yaml \
--weights yolov5x.pt \ # Model name
--save_period 1\ # Save model after interval
--project kaggle-siim-covid # W&B project name

In [ ]:
!python train.py --img {IMG_SIZE} \
                 --batch {8} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --cfg models/yolov5x.yaml \
                 --weights yolov5x.pt \
                 --save_period 1\
                 --project kaggle-siim-covid-2

train: weights=yolov5x.pt, cfg=models/yolov5x.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=35, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=kaggle-siim-covid-2, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=1, artifact_alias=latest, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-304-g2c073cd torch 1.9.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160.5MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0

In [ ]:
TEST_PATH = '/content/test/'

In [ ]:
MODEL_PATH = '/content/yolov5/kaggle-siim-covid-2/exp2/weights/best.pt'

In [ ]:
--weights {MODEL_PATH} \ # path to the best model.
--source {TEST_PATH} \ # absolute path to the test images.
--img {IMG_SIZE} \ # Size of image
--conf 0.001 \ # Confidence threshold (default is 0.25)
--iou 0.5 \ # IOU threshold (default is 0.45)
--max-det 3 \ # Number of detections per image (default is 1000) 
--save-txt \ # Save predicted bounding box coordinates as txt files
--save-conf # Save the confidence of prediction for each bounding box

In [220]:
!python detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.2 \
                  --max-det 3 \
                  --iou 0.5 \
                  --save-txt \
                  --save-conf \
                  --augment

detect: weights=['/content/yolov5/kaggle-siim-covid-2/exp2/weights/best.pt'], source=/content/test/, imgsz=640, conf_thres=0.2, iou_thres=0.5, max_det=3, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-304-g2c073cd torch 1.9.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160.5MB)

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 476 layers, 87205423 parameters, 0 gradients

In [225]:
PRED_PATH = '/content/yolov5/runs/detect/exp2/labels'
!ls {PRED_PATH}

00fc8fc35dc1.txt  4041be34745f.txt  86734b7cc20f.txt  bf8e59f80d5c.txt
02eceb0fc405.txt  404d53460643.txt  8757ce9b582c.txt  c010822b2938.txt
03a778f5a68b.txt  409322480a9b.txt  875cf71bd22a.txt  c02ef6a70c7e.txt
042979d394a1.txt  413344e1032c.txt  877b017a6d20.txt  c0a746ef9f9f.txt
04396dd5af2f.txt  41c25ec8c44c.txt  87a0829f53c1.txt  c0f5131b1c99.txt
04720d7e4e42.txt  43e370c03cd2.txt  87c51db67bf7.txt  c263b1e9aa64.txt
049fce8128f9.txt  445298de90b9.txt  87d8baf120a6.txt  c28570a073d9.txt
04a083828fec.txt  458417ad6155.txt  8829d22c3e2b.txt  c34113e0228a.txt
04de800ea41a.txt  47a976677416.txt  8857adac8318.txt  c39146cbda47.txt
04f4a36c08a2.txt  47ea205fd755.txt  88782677cbec.txt  c3c99fea8c77.txt
05435da60872.txt  4810ae5dbc08.txt  88b9ca325a4c.txt  c3d79bd7e833.txt
0562436fde50.txt  4905b651a091.txt  88debce8e1c3.txt  c3deaf98bf56.txt
059eae61ad15.txt  49c1b06028a1.txt  89015a2258aa.txt  c3ded8550e66.txt
05a70a1c16c1.txt  49e9379c42dc.txt  8927235865c9.txt  c43e88a44468.txt
05feea

In [224]:
%cat /content/yolov5/runs/detect/exp2/labels/ffb8115a304c.txt

1 0.717188 0.438281 0.171875 0.329688 0.282072
1 0.317969 0.382031 0.204688 0.457812 0.310724


In [226]:
prediction_files = os.listdir(PRED_PATH)
print('Number of test images predicted as opaque: ', len(prediction_files))

Number of test images predicted as opaque:  814


In [ ]:
#submission
# The submisison requires xmin, ymin, xmax, ymax format. 
# YOLOv5 returns x_center, y_center, width, height
def correct_bbox_format(bboxes):
    correct_bboxes = []
    for b in bboxes:
        xc, yc = int(np.round(b[0]*IMG_SIZE)), int(np.round(b[1]*IMG_SIZE))
        w, h = int(np.round(b[2]*IMG_SIZE)), int(np.round(b[3]*IMG_SIZE))

        xmin = xc - int(np.round(w/2))
        xmax = xc + int(np.round(w/2))
        ymin = yc - int(np.round(h/2))
        ymax = yc + int(np.round(h/2))
        
        correct_bboxes.append([xmin, xmax, ymin, ymax])
        
    return correct_bboxes

# Read the txt file generated by YOLOv5 during inference and extract 
# confidence and bounding box coordinates.
def get_conf_bboxes(file_path):
    confidence = []
    bboxes = []
    with open(file_path, 'r') as file:
        for line in file:
            preds = line.strip('\n').split(' ')
            preds = list(map(float, preds))
            confidence.append(preds[-1])
            bboxes.append(preds[1:-1])
    return confidence, bboxes

In [227]:
# Read the submisison file
sub_df = pd.read_csv('/content/sample_submission.csv')
sub_df.tail()

,id,PredictionString
2472,46719b856de1_image,none 1 0 0 1 1
2473,31c07523a69a_image,none 1 0 0 1 1
2474,f77d7d1aebab_image,none 1 0 0 1 1
2475,ccc5b63ca96d_image,none 1 0 0 1 1
2476,5e8ac1fe2b82_image,none 1 0 0 1 1


In [228]:
predictions = []

for i in tqdm(range(len(sub_df))):
    row = sub_df.loc[i]
    id_name = row.id.split('_')[0]
    id_level = row.id.split('_')[-1]
    
    if id_level == 'study':
        # do study-level classification
        predictions.append("Negative 1 0 0 1 1") # dummy prediction
        
    elif id_level == 'image':
        # we can do image-level classification here.
        # also we can rely on the object detector's classification head.
        # for this example submisison we will use YOLO's classification head. 
        # since we already ran the inference we know which test images belong to opacity.
        if f'{id_name}.txt' in prediction_files:
            # opacity label
            confidence, bboxes = get_conf_bboxes(f'{PRED_PATH}/{id_name}.txt')
            bboxes = correct_bbox_format(bboxes)
            pred_string = ''
            for j, conf in enumerate(confidence):
                pred_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '
            predictions.append(pred_string[:-1]) 
        else:
            predictions.append("None 1 0 0 1 1")

100%|██████████| 2477/2477 [00:00<00:00, 5442.36it/s]


In [229]:
sub_df['PredictionString'] = predictions
sub_df.to_csv('submission3.csv', index=False)
sub_df.tail()

,id,PredictionString
2472,46719b856de1_image,None 1 0 0 1 1
2473,31c07523a69a_image,opacity 0.283913 158 286 190 348 opacity 0.406...
2474,f77d7d1aebab_image,opacity 0.2068 498 590 172 356
2475,ccc5b63ca96d_image,None 1 0 0 1 1
2476,5e8ac1fe2b82_image,opacity 0.359283 393 587 270 618 opacity 0.444...


In [231]:
!cp /content/yolov5/submission3.csv {EXPERIMENT_DIR}

In [ ]:
!cp {MODEL_PATH} {EXPERIMENT_DIR}